In [231]:
import numpy as np
vecs = list(map(int, np.random.random(300) * 100))
vecs = np.resize(vecs, (100, 3))
vecs = np.insert(vecs, 3, np.random.random(100) * 10, axis=1)

In [232]:
def rotate_vec(vec, theta=None, i=None, j=None):

    dim = vec.size
    if theta is None:
        theta = 2 * np.pi * np.random.random()

    if i is None:
        i = np.random.choice(dim)
    if j is None:
        j = np.random.choice(dim)
        while i == j:
            j = np.random.choice(dim)
    rotational_matrix = np.identity(dim)
    rotational_matrix[i][i] = rotational_matrix[j][j] = np.cos(theta)
    rotational_matrix[i][j] = np.sin(theta)
    rotational_matrix[j][i] = -np.sin(theta)

    return np.matmul(rotational_matrix, vec), rotational_matrix

In [233]:
dim = 4
theta = np.pi / 2
i = 0
j = 1

In [234]:
np.set_printoptions(precision=2)
rotate_vec(vecs[0], theta= 2 * np.pi * np.random.random())

(array([11.16, 56.9 , 60.  ,  2.  ]),
 array([[ 0.69, -0.73,  0.  ,  0.  ],
        [ 0.73,  0.69,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  1.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  1.  ]]))

Making the auto-encoder

In [235]:
tot_rot = np.identity(4)
for i in range(3):
    _, rotvect = rotate_vec(np.zeros(4))
    tot_rot = np.matmul(rotvect, tot_rot)
    vecs = [np.matmul(rotvect, vec) for vec in vecs]

tot_rot


array([[ 0.21,  0.  , -0.91, -0.36],
       [-0.35, -0.93, -0.07, -0.03],
       [-0.91,  0.36, -0.18, -0.07],
       [ 0.  ,  0.  , -0.37,  0.93]])